In [1]:
import os, sys, json, socket, time, signal, glob, csv, math, subprocess, re, shutil
from typing import List, Optional
import numpy as np
import random

import pathlib
import logging
import threading
import queue

import pandas as pd
import time

import pickle

# Models that our oracle needs
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RepeatedKFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import RadiusNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import StackingRegressor
from matplotlib import pyplot
import seaborn as sns
import xgboost as xgb

# place were the json file is
arquivoSunshineConfig = r"C:\msys64\home\MoonlightServer\sunshine\build\encoder_runtime.json"


import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [2]:
oldBitrate = 10000
oldFPS = 60

def setaBitrateFPS(bitrate,fps):
    global oldBitrate
    global oldFPS
    bit = int(round(bitrate))
    efipesi = int(round(fps))
    if((oldBitrate == bit) and (oldFPS == efipesi)):
        return
    efipesi = efipesi * 2
    escrever = "{ \"bitrate_kbps\": "+str(bit)+", \"fps\": 120, \"cap_fps\": "+str(efipesi)+", \"fps_cap_mode\": \"drop\" }"
    with open(arquivoSunshineConfig, 'w') as f:
        f.write(escrever)
    oldBitrate = bit
    oldFPS = efipesi

def iniciaBitrateFPS():
    escrever = "{ \"bitrate_kbps\": 10000, \"fps\": 120, \"cap_fps\": 120, \"fps_cap_mode\": \"drop\" }"#drop ou sleep
    with open(arquivoSunshineConfig, 'w') as f:
        f.write(escrever)

In [3]:
def transform_minmax_MEU(X, clip=True):

    X = np.asarray(X, dtype=np.float32)
    mins = np.asarray([0.0, 0.0, 0.0, 0.0, 0.0, 0.0], dtype=np.float32)#[0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    maxs = np.asarray([150.0, 150.0, 25.0, 25.0, 3.0, 3.0], dtype=np.float32)#[150.0, 150.0, 25.0, 25.0, 3.0, 3.0]

    if len(X[0]) != len(mins):
        raise ValueError(f"X tem {len(X[0])} colunas, mas o normalizador foi treinado com {len(mins)}.")

    denom = maxs - mins
    Xn = (X - mins) / denom
    if clip:
        Xn = np.clip(Xn, 0.0, 1.0)
    return Xn.astype(np.float32)

def prepare(arr):
    arr = np.asarray(arr, dtype=float)

    if np.nanmin(arr) >= 0.0 and np.nanmax(arr) <= 1.0:
        arr = arr * 6.0

    arr = np.clip(arr, 0.0, 6.0)

    return arr


def loadModel(nome):
    # save the model to disk
    filename = nome+".sav"
    loaded_model = pickle.load(open(filename, 'rb'))
    return loaded_model

def fazerPredicaoDe1(valoresX):
    Xtest = []
    Xtest.append(np.nan_to_num(valoresX))
    Xtest = transform_minmax_MEU(Xtest)
    model = loadModel("Melhor_Modelo_Stacking-cut-7-com-MLP")
    pred = model.predict(Xtest).flatten()
    y_pred = prepare(pred)
    return y_pred[0]  

In [ ]:
import SubjectSieve


cfg = SubjectSieve.ControllerConfig()
ctrl = SubjectSieve.QoEOnlyPensieveLikeController(cfg)
ctrl.set_encoder_state(bitrate_kbps=10000, fps=60)


def callController(X):
    Xtest = []
    Xtest.append(np.nan_to_num(X))
    QoE = fazerPredicaoDe1(Xtest[0])
    obs = {
      "QoE": QoE
    }
    tgt_bitrate, tgt_fps = ctrl.update(obs)
    setaBitrateFPS(tgt_bitrate, tgt_fps)
    ctrl.confirm_apply(tgt_bitrate, tgt_fps)
    return tgt_bitrate, tgt_fps, QoE


def goodNetwork():
    for i in range(1000):
        obs = {
          "QoE": 6
        }
        tgt_bitrate, tgt_fps = ctrl.update(obs)
        ctrl.confirm_apply(tgt_bitrate, tgt_fps)



# Get network information, then pass to callController that will use the oracle to predict QoE and run SubjectSieve
# Our QoE oracle expects network metrics in this order:
# Video Delay, Input Delay, Video Jitter, Input Jitter, Video Loss, Input Loss

Xtest = []
Xtest.append(80)#   Video Delay
Xtest.append(15)#   Input Delay
Xtest.append(10)#   Video Jitter
Xtest.append(5)#    Input Jitter
Xtest.append(0.4)#  Video Loss
Xtest.append(0.1)#  Input Loss
      
tgt_bitrate, tgt_fps, QoE = callController(Xtest)

print("QOE: "+str(QoE))
print("tgt_bitrate: "+str(tgt_bitrate)+", tgt_fps: "+str(tgt_fps))